In [ ]:
import sys
sys.path.append("C:/Users/user/meepc")
import numpy as np
import pandas as pd
from models import Hankel,RobustPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def calculate_fpr(y_actual, y_prediction):
    fp = 0
    tn = 0
    
    for i in range(len(y_actual)):
        if y_actual[i] == 0 and y_prediction[i] == 1:
            fp += 1
        elif y_actual[i] == 0 and y_prediction[i] == 0:
            tn += 1
    fpr= fp/(fp+tn)
    return fpr

In [ ]:
train_normal = pd.read_csv('~/data/te/normal_training.csv')
train_attack = pd.read_csv('~/data/te/attack_training.csv')
train_normal = pd.concat((train_normal,train_attack),axis=0,ignore_index=True)
y_truth = train_normal.loc[:,'LABEL']

In [ ]:
hankel = Hankel()
lag = 500
stride = 0.5

In [ ]:
#attack indexes in original training data
temp = hankel.fit(np.array(y_truth) ,lag,stride)
attack_idx = np.any(temp>0,axis=0).astype(int)
attack_idx = np.where(attack_idx>0)[0]
attack_idx

In [ ]:
sensors = [col for col in train_normal.columns if col not in ['faultNumber', 'simulationRun', 'sample','LABEL']]

In [ ]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))
X_attack = pd.DataFrame(data=scaler.fit_transform(train_attack.loc[:,sensors]),index=train_attack.index,columns=sensors).reset_index().drop(columns=['index'])

In [ ]:
df_test1 = pd.read_csv('~/data/te/normal_testing.csv')
df_test1 = df_test1.sample(n=15000, random_state=42)
df_test1.reset_index(drop=True, inplace=True)
df_test2 = pd.read_csv('~/data/te/attack_testing.csv')
df_test2 = df_test2.groupby('faultNumber').apply(lambda x: x.sample(n=150, random_state=42))
df_test2.reset_index(drop=True, inplace=True)
df_test=pd.concat((df_test1,df_test2),axis=0)

# Epasad with 1 cluster

In [ ]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_test = df_test.loc[:,'LABEL']

In [ ]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [ ]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
fpr = []
cluster_indexes=[]
pca_counts=0
meepc_counts=0

for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = RobustPipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,kscore_init='inertia',tune=False,alpha=0.15,y_truth=y_truth)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    fpr.append(calculate_fpr(y_actual,y_predicted))
    sensor_models.append(model)    
    #clustering
    cluster_indexes.append(model.cluster_idx)
    #pca

    pca_counts=max(pca_counts,model.total_pca_attacks)
    
    #meepc
    meepc_counts=max(meepc_counts,model.counts_meepc)

In [ ]:
cluster_inactive_idx = np.unique(np.concatenate(cluster_indexes))
commonidxs=np.intersect1d(attack_idx,cluster_inactive_idx)
percentage = len(commonidxs)/len(attack_idx)*100
print(percentage)

94.11764705882352
[418 419 420 421 430 431 432 442 443 444 445 446 462 463 464 465 466 489
 490 491 492 498 499 500 501 502 503 507 508 509 510 511]


In [ ]:
percentage = pca_counts/len(attack_idx)*100
print(percentage)

47.05882352941176


In [ ]:
percentage = meepc_counts/len(attack_idx)*100
print(percentage) 

32.35294117647059


In [ ]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))
print("False Positive Rate ",calculate_fpr(y_actual,y_predicted))

Accuracy  0.4090909090909091
Precision  0.4326923076923077
Recall  0.8823529411764706
F1-score  0.5806451612903226
False Positive Rate  1.0


In [ ]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.5833916083916084 Median : 0.6181818181818182 Min : 0.18181818181818182 Max : 0.7090909090909091
precision - Mean: 0.9057995891299253 Median : 1.0 Min : 0.24675324675324675 Max : 1.0
recall -    Mean: 0.24019607843137256 Median : 0.23529411764705882 Min : 0.058823529411764705 Max : 0.6078431372549019
f1 -        Mean: 0.3402945673555318 Median : 0.3278688524590164 Min : 0.1111111111111111 Max : 0.5555555555555556


# Multiple clusters + No threshold tuning

In [ ]:
cluster_accuracy = []
cluster_precision = []
cluster_recall = []
cluster_fscore = []
cluster_fpr = []
for k in range(2,6):
    print('-------------------------------------',k,'-------------------------------------')
    
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    cluster_indexes=[]
    pca_counts=0
    meepc_counts=0
    for i,sens in enumerate(sensors):
        print(sens)
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = RobustPipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia',alpha=0.15,y_truth=y_truth)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model) 
        cluster_indexes.append(model.cluster_idx)
        pca_counts=max(pca_counts,model.total_pca_attacks)
        meepc_counts=max(meepc_counts,model.counts_meepc)
           
    #cluster
    cluster_inactive_idx = np.unique(np.concatenate(cluster_indexes))
    commonidxs=np.intersect1d(attack_idx,cluster_inactive_idx)
    percentage = len(commonidxs)/len(attack_idx)*100
    print("Cluster - attack points detected {} %".format(percentage))
    #PCA
    
    percentage = pca_counts/len(attack_idx)*100
    print("PCA - attack points detected {} %".format(percentage))
    #Meepc
    percentage = meepc_counts/len(attack_idx)*100
    print("Meepc - attack points detected {} %".format(percentage),"/n")
            
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy.append(accuracy_score(y_actual, y_predicted))
    cluster_precision.append(precision_score(y_actual,y_predicted))
    cluster_recall.append(recall_score(y_actual,y_predicted))
    cluster_fscore.append(f1_score(y_actual,y_predicted))
    cluster_fpr.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy[-1])
    print("Precision ",cluster_precision[-1])
    print("Recall ",cluster_recall[-1])
    print("F1-score ",cluster_fscore[-1])
    print("False Positive Rate ",cluster_fpr[-1],"\n")

------Number of Clusters:  2 -----------
Accuracy  0.9142857142857143
Precision  0.6470588235294118
Recall  1.0
F1-score  0.7857142857142858
False Positive Rate  0.1016949152542373 

------Number of Clusters:  3 -----------
Accuracy  1.0
Precision  1.0
Recall  1.0
F1-score  1.0
False Positive Rate  0.0 

------Number of Clusters:  4 -----------
Accuracy  1.0
Precision  1.0
Recall  1.0
F1-score  1.0
False Positive Rate  0.0 

------Number of Clusters:  5 -----------
Accuracy  0.9857142857142858
Precision  1.0
Recall  0.9090909090909091
F1-score  0.9523809523809523
False Positive Rate  0.0 

------Number of Clusters:  6 -----------
Accuracy  0.9714285714285714
Precision  1.0
Recall  0.8181818181818182
F1-score  0.9
False Positive Rate  0.0 

------Number of Clusters:  7 -----------
Accuracy  0.9714285714285714
Precision  1.0
Recall  0.8181818181818182
F1-score  0.9
False Positive Rate  0.0 



In [ ]:
print("Accuracy Scores: ",cluster_accuracy)
print("Precision Scores: ",cluster_precision)
print("Recall Scores: ",cluster_recall)
print("F1 Scores: ",cluster_fscore)
print("False Positive Rates: ",cluster_fpr)

Accuracy Scores:  [0.9142857142857143, 1.0, 1.0, 0.9857142857142858, 0.9714285714285714, 0.9714285714285714]
Precision Scores:  [0.6470588235294118, 1.0, 1.0, 1.0, 1.0, 1.0]
Recall Scores:  [1.0, 1.0, 1.0, 0.9090909090909091, 0.8181818181818182, 0.8181818181818182]
F1 Scores:  [0.7857142857142858, 1.0, 1.0, 0.9523809523809523, 0.9, 0.9]
False Positive Rates:  [0.1016949152542373, 0.0, 0.0, 0.0, 0.0, 0.0]
